In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import time
import os
from tqdm import trange

In [2]:

def search_results(search):
#     Search the keyword on the website and press enter
    driver.find_element_by_xpath('//*[@id="search"]').clear()
    driver.find_element_by_xpath('//*[@id="search"]').send_keys(search)
    driver.find_element_by_xpath('//*[@id="search"]').send_keys(Keys.ENTER)

def get_links(search,pages = 1000):
#   get_links does the search query and collects all the links to the wallpapers till the end of the pages.
    driver.get('https://wall.alphacoders.com')
    search_results(search)
    time.sleep(2)
    url = driver.current_url

    wait = WebDriverWait(driver,5)

    links = []
    prev_len= new_len = 0
    i=1
    flag = 1
    while((flag) & (i<=pages)):
        try:
#             Get all the hrefs from the page
            elems = driver.find_elements_by_xpath("//a[@href]")
            for elem in elems:
                link = elem.get_attribute("href")
#                 filter the links that specify a certain format to get the wallpaper links
                if ('i=' in link) & ('//wall.alphacoders.com' in link):
                    links.append(link)
            new_len = len(set(links))
#             If no new links were found break the loop 
            if new_len==prev_len:
                flag = 0
            prev_len = len(set(links))
            i+=1
            driver.get(url+'&page='+str(i))
            time.sleep(5)
        except:
            print('Total Pages Found %i' %i)
            break
    return links

def download_links(links):
#     Once all links are scraped, download all these links in their original sizes.
    links = list(set(links))
    error_log = []
    for i in trange(len(links)):
        try:
            driver.get(links[i])
            time.sleep(3)
            wait.until(lambda driver: driver.find_element_by_xpath('//*[@id="page_container"]/div[7]/div[1]/span'))
            element = driver.find_element_by_xpath('//*[@id="page_container"]/div[7]/div[1]/span')
            driver.execute_script("arguments[0].scrollIntoView();", element)
            element.click()
            time.sleep(3)
        except:
            error_log.append(links[i])
    driver.quit()

In [3]:
######################## User inputs here #######################
searches = [
    'Peaky Blinders','Gal Gadot',
]
download_path = r"C:\Users\abhij\Downloads\Wallpapers"
chromedriver = 'C:/Users/abhij/Downloads/Programs/chromedriver.exe'

In [ ]:
for search in searches:
    try:
        print('Downloading %s' %search)
        try:
#             Create a folder to store all the downloads there
            os.mkdir(os.path.join(download_path ,search))
        except:
            print('Folder already exists !!!')
        
        chromeOptions = webdriver.ChromeOptions()
#         Update the download directory of the driver
        prefs = {"download.default_directory" : os.path.join(download_path ,search)}
        chromeOptions.add_experimental_option("prefs",prefs)
        driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
        driver.maximize_window()
#         Add pages argument below to limit the number of page searches
        links = get_links(search)
        wait = WebDriverWait(driver,10)
        download_links(links)
    except:
        time.sleep(50)
        